In [54]:
import numpy as np
import pandas as pd
import pathlib
import os
from decimal import *
getcontext().prec = 5
pd.set_option("display.precision", 3)

df_probabilities = {
	6: pd.read_csv('vars\\prob_06.csv', header=None),
	12: pd.read_csv('vars\\prob_12.csv', header=None)
}

df_tables = {
	6: pd.read_csv('vars\\table_06.csv', header=None),
	12: pd.read_csv('vars\\table_12.csv', header=None)
}

lengths = {
	6: {
		'text_count': len(df_tables[6].columns),
		'key_count': len(df_tables[6])
	},
	12: {
		'text_count': len(df_tables[12].columns),
		'key_count': len(df_tables[12])
	}
}


In [55]:
def get_cipher_text_unconditional_probability(variant, cipher_text):
	df_table = df_tables[variant]
	df_prob = df_probabilities[variant]
	
	probability = 0.0

	for plain_text_index in range(lengths[variant]['text_count']):
		for key_text_index in range(lengths[variant]['key_count']):
			if (df_table[plain_text_index][key_text_index] == cipher_text):
				plain_text_probability = df_prob[plain_text_index][0]
				key_probability = df_prob[key_text_index][1]

				probability += plain_text_probability * key_probability
	
	return probability

df_cipher_texts_unconditional_prob = {
	6: np.array([get_cipher_text_unconditional_probability(6, i) for i in range(lengths[6]['text_count'])]),
	12: np.array([get_cipher_text_unconditional_probability(12, i) for i in range(lengths[12]['text_count'])])
}


In [56]:
def cipher_and_plain_texts_common_probability(variant, cipher_text, plain_text):
	df_table = df_tables[variant]
	df_prob = df_probabilities[variant]
	
	probability = 0.0

	for key_index in range(lengths[variant]['key_count']):
		if (df_table[plain_text][key_index] == cipher_text):
			plain_text_probability = df_prob[plain_text][0]
			key_probability = df_prob[key_index][1]

			probability += plain_text_probability * key_probability
	
	return probability

df_cipher_texts_common_prob = {
	6: np.array([[cipher_and_plain_texts_common_probability(6, i, j) for j in range(lengths[6]['text_count'])] for i in range(lengths[6]['text_count'])]),
	12: np.array([[cipher_and_plain_texts_common_probability(12, i, j) for j in range(lengths[12]['text_count'])] for i in range(lengths[12]['text_count'])])
}


In [57]:
def cipher_and_plain_texts_conditional_probability(variant):
    return np.array([
			[df_cipher_texts_common_prob[variant][i][j] / df_cipher_texts_unconditional_prob[variant][i] for j in range(lengths[6]['text_count'])
		] for i in range(lengths[6]['text_count'])])

df_cipher_texts_conditional_prob = {
	6: cipher_and_plain_texts_conditional_probability(6),
	12: cipher_and_plain_texts_conditional_probability(12)
}


In [58]:
pd.DataFrame(df_cipher_texts_conditional_prob[6])

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0.000,0.040,0.000,0.040,0.000,0.040,0.000,0.280,0.040,0.040,0.040,0.080,0.040,0.080,0.040,0.040,0.040,0.000,0.080,0.080
1,0.000,0.000,0.240,0.080,0.000,0.080,0.040,0.080,0.080,0.040,0.000,0.000,0.000,0.000,0.040,0.160,0.080,0.040,0.000,0.040
2,0.200,0.033,0.000,0.233,0.033,0.000,0.000,0.067,0.000,0.033,0.000,0.033,0.100,0.100,0.033,0.067,0.000,0.000,0.033,0.033
3,0.200,0.000,0.133,0.000,0.033,0.067,0.000,0.067,0.033,0.033,0.033,0.000,0.000,0.233,0.033,0.000,0.000,0.100,0.033,0.000
4,0.000,0.040,0.040,0.000,0.000,0.040,0.120,0.040,0.040,0.080,0.360,0.000,0.080,0.000,0.000,0.000,0.000,0.120,0.040,0.000
5,0.200,0.033,0.000,0.000,0.000,0.000,0.133,0.033,0.033,0.000,0.000,0.033,0.033,0.000,0.067,0.067,0.000,0.033,0.133,0.200
6,0.450,0.025,0.000,0.000,0.000,0.000,0.000,0.050,0.050,0.025,0.000,0.025,0.025,0.000,0.050,0.025,0.100,0.150,0.000,0.025
7,0.200,0.033,0.067,0.000,0.067,0.033,0.200,0.000,0.033,0.000,0.067,0.033,0.033,0.000,0.033,0.000,0.033,0.033,0.067,0.067
8,0.000,0.000,0.000,0.040,0.320,0.040,0.000,0.040,0.000,0.000,0.040,0.040,0.040,0.040,0.120,0.080,0.040,0.080,0.000,0.080
9,0.000,0.080,0.040,0.040,0.000,0.000,0.160,0.000,0.320,0.000,0.000,0.080,0.000,0.160,0.040,0.040,0.000,0.000,0.000,0.040


In [59]:
pd.DataFrame(df_cipher_texts_conditional_prob[12])

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0.000,0.233,0.000,0.033,0.000,0.033,0.067,0.300,0.000,0.033,0.000,0.067,0.067,0.000,0.033,0.033,0.000,0.067,0.033,0.000
1,0.000,0.000,0.140,0.080,0.120,0.000,0.140,0.000,0.040,0.000,0.040,0.040,0.000,0.120,0.040,0.040,0.000,0.040,0.080,0.080
2,0.117,0.117,0.000,0.000,0.033,0.067,0.000,0.000,0.033,0.033,0.033,0.100,0.000,0.117,0.000,0.217,0.100,0.033,0.000,0.000
3,0.406,0.090,0.103,0.000,0.000,0.026,0.000,0.052,0.026,0.116,0.052,0.000,0.052,0.026,0.026,0.000,0.026,0.000,0.000,0.000
4,0.233,0.000,0.000,0.033,0.000,0.033,0.067,0.067,0.150,0.033,0.033,0.117,0.033,0.033,0.033,0.033,0.033,0.000,0.033,0.033
5,0.100,0.300,0.000,0.000,0.000,0.000,0.086,0.029,0.029,0.057,0.129,0.000,0.029,0.029,0.029,0.029,0.029,0.100,0.029,0.000
6,0.108,0.215,0.031,0.031,0.031,0.000,0.000,0.031,0.062,0.000,0.031,0.108,0.031,0.169,0.000,0.000,0.031,0.092,0.031,0.000
7,0.090,0.406,0.000,0.026,0.219,0.000,0.000,0.000,0.052,0.052,0.026,0.077,0.000,0.000,0.000,0.000,0.026,0.026,0.000,0.000
8,0.000,0.000,0.000,0.080,0.000,0.000,0.140,0.000,0.000,0.040,0.180,0.040,0.080,0.120,0.000,0.120,0.000,0.120,0.080,0.000
9,0.000,0.000,0.040,0.140,0.040,0.080,0.080,0.000,0.040,0.000,0.040,0.000,0.040,0.000,0.000,0.080,0.220,0.040,0.080,0.080


In [60]:
def get_bayes_decision_function(variant, cipher_text):
	return np.argmax(df_cipher_texts_conditional_prob[variant][cipher_text])

df_bayes_decision_function = {
	6: [get_bayes_decision_function(6, i) for i in range(lengths[6]['text_count'])],
	12: [get_bayes_decision_function(12, i) for i in range(lengths[6]['text_count'])]
}


In [61]:
pd.DataFrame({ 'C': range(lengths[6]['text_count']), 'M': df_bayes_decision_function[6] }).transpose()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
C,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
M,7,2,3,13,10,0,0,0,4,8,0,0,1,14,0,12,18,0,9,0


In [62]:
pd.DataFrame({ 'C': range(lengths[12]['text_count']), 'M': df_bayes_decision_function[12] }).transpose()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
C,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
M,7,2,15,0,0,1,1,1,10,16,12,0,1,19,0,5,0,14,3,14


In [63]:
def get_bayes_function_losses(variant, cipher_text, plain_text):
	return 0 if df_bayes_decision_function[6][cipher_text] == plain_text else 1

def get_bayes_function_average_losses(variant):
	return sum(
		[get_bayes_function_losses(variant, i, j) * df_cipher_texts_common_prob[variant][i][j] for j in range(lengths[variant]['text_count']) for i in range(lengths[variant]['text_count'])]
	)

df_bayes_function_average_losses = {
	6: get_bayes_function_average_losses(6),
	12: get_bayes_function_average_losses(12)
}


In [64]:
print(df_bayes_function_average_losses[6])
print(df_bayes_function_average_losses[12])


0.6703999999999999
0.8780000000000021


In [65]:
def get_stochastic_decision_function(variant):
	matrix = np.zeros((lengths[variant]['text_count'], lengths[variant]['text_count']))

	for i in range(lengths[variant]['text_count']):
		current_probability_set = df_cipher_texts_conditional_prob[variant][i]

		max_probability = max(current_probability_set)
		max_probability_entrance = list(current_probability_set).count(max_probability)

		for j in range(lengths[variant]['text_count']):
			matrix[i][j] = 1 / max_probability_entrance if current_probability_set[j] == max_probability else 0

	return matrix

df_stochastic_decision_function = {
	6: get_stochastic_decision_function(6),
	12: get_stochastic_decision_function(12)
}


In [66]:
pd.DataFrame(df_stochastic_decision_function[6])

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5
6,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.5,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [67]:
pd.DataFrame(df_stochastic_decision_function[12])

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.5,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [69]:
def get_stochastic_function_losses(variant, cipher_text, plain_text):
	return sum([df_stochastic_decision_function[variant][cipher_text][i] for i in range(lengths[variant]['text_count']) if i != plain_text])

def get_stochastic_function_average_losses(variant):
	return sum(
		[get_stochastic_function_losses(variant, i, j) * df_cipher_texts_common_prob[variant][i][j] for j in range(lengths[variant]['text_count']) for i in range(lengths[variant]['text_count'])]
	)

df_stochastic_function_average_losses = {
	6: get_stochastic_function_average_losses(6),
	12: get_stochastic_function_average_losses(12)
}


In [70]:
print(df_stochastic_function_average_losses[6])
print(df_stochastic_function_average_losses[12])

0.6703999999999999
0.7412000000000012
